In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import glob
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_csv = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
train_csv.head()

In [ ]:
train_csv.drop_duplicates(subset="dataset_label",inplace=True)

In [ ]:
train = train_csv.copy()

In [ ]:
def clean_sentence(txt):
     return re.sub('[^A-Za-z0-9.]+', ' ', str(txt).lower())   

In [ ]:
import nltk
DATA = []
ent_count = 0
empty_count = 0

for idx,row in tqdm(train.iterrows()):
    pub = "../input/coleridgeinitiative-show-us-the-data/train/" + row.Id + ".json"            
    f = open(pub)  
    data = json.load(f)      
    
    balanced = False

    sentences = nltk.tokenize.sent_tokenize(str(data))

    for sentence in sentences:          
        sentence = clean_sentence(sentence).strip()        
        a = re.search(row.dataset_label.lower(),sentence)            
        if  a != None:
            DATA.append((sentence,{"entities":[(a.span()[0],a.span()[1],"DATASET")]}))
            ent_count = ent_count + 1
            balanced = True
        else:
            if balanced:
                DATA.append((sentence,{"entities":[]}))
                empty_count = empty_count + 1
                balanced = False
print("Text with entities:",ent_count,"Text without entities:",empty_count)

In [ ]:
len(DATA)

In [ ]:
TRAIN_DATA = DATA

In [ ]:
import random
import spacy
from spacy.util import minibatch, compounding
from pathlib import Path

def train_model_spacy(TRAIN_DATA, iterations, model):
    print(f"downloads = {model}")
    if model is not None:
        print("Model is existing!")
        nlp=spacy.load(model)
        print("Model is loaded")
    else:
        print("Creating new model")
        nlp=spacy.blank('en')
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe('ner')
    
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    #disable other pipes
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        if model is None:
            optimizer = nlp.begin_training()
        else:
            optimizer = nlp.entity.create_optimizer()
        tags = dict()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 16.0, 1.001))
            # type 2 with mini batch
            for batch in batches:                
                texts, annotations = zip(*batch)                
                golds = annotations 
                nlp.update(
                    texts,  # batch of texts
                    golds,  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data                    
                    losses=losses,
                    sgd=optimizer
                )
            print(losses)
    return nlp

In [ ]:
model = train_model_spacy(TRAIN_DATA,15,"en_core_web_sm")

In [ ]:
test_pubs = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv").Id
test_pubs

# load submission.csv
sub = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")

In [ ]:
i = 0

for pub in test_pubs:    
    print("pub:",pub)
    
    f = open("../input/coleridgeinitiative-show-us-the-data/test/" + pub + ".json")  
    
    predicted_text = ""
    
    data = json.load(f)      

    sentences = nltk.tokenize.sent_tokenize(str(data))

    for sentence in sentences:          
        sentence = clean_sentence(sentence).strip()        
        doc = model(sentence)
        for ent in doc.ents:
            predicted_text = predicted_text + ent.text + "|"
    
            #print("pub:",pub, "\n",predicted_text[:-1].strip(),"\n")

    print("final:",predicted_text[:-1])
    sub.PredictionString.loc[i] = predicted_text[:-1].strip()
    
    i = i + 1

In [ ]:
sub.to_csv('submission.csv',index=False)
sub